In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth',None)   #this displays the dataframe in full width
import collections
from collections import Counter
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import emoji
from indicnlp.tokenize import indic_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer #Data transformation
from sklearn.model_selection import train_test_split #Data testing
from sklearn.linear_model import LogisticRegression #Prediction Model
from sklearn.metrics import accuracy_score, classification_report #Comparison between real and predicted

import matplotlib
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt



[nltk_data] Downloading package punkt to /Users/shraddha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_sarcastic = pd.read_csv('/Users/shraddha/Documents/UEL/MYDES/Hindi_Tweets-SARCASTIC.csv')
df_non_sarcastic = pd.read_csv('/Users/shraddha/Documents/UEL/MYDES/Hindi_Tweets-NON-SARCASTIC.csv')
df_sarcastic['label'] = 'sarcastic'
df_non_sarcastic['label'] = 'non_sarcastic'
df = pd.concat([df_sarcastic, df_non_sarcastic], axis=0)
df = df.drop(['username','acctdesc','location','following','followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts', 'retweetcount', 'hashtags'] ,axis=1)
df = df.reset_index()
df = df.drop('index',axis=1)
df.head(100)

In [4]:
# create a function called count_length() which will count the number of words in the text.
def count_length():
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" ")))
count_length()
df.head()

text  \
0  हमारे गांव में एक मास्टर जी रहते थे, वो मोहल्ले के बच्चों को अपने स्कूल आने को कहते थे। जब मैं बड़ा हुआ तो पता चला- स्कूल मास्टर के बेटे का, बस उनके भतीजे की, कैंटीन उनके भांजे की और  पढ़ाने वाले भी सब उनके रिश्तेदार ही थे..! \n\nआज मास्टर जी में सरकार में मंत्री हैं.! \n\n#कटाक्ष   
1  हमारे गांव में एक मास्टर जी रहते थे, वो मोहल्ले के बच्चों को अपने स्कूल आने को कहते थे। जब मैं बड़ा हुआ तो पता चला- स्कूल मास्टर के बेटे का, बस उनके भतीजे की, कैंटीन उनके भांजे की और  पढ़ाने वाले भी सब उनके रिश्तेदार ही थे..! \n\nआज मास्टर जी में सरकार में मंत्री हैं.! \n\n#कटाक्ष   
2  हमारे गांव में एक मास्टर जी रहते थे, वो मोहल्ले के बच्चों को अपने स्कूल आने को कहते थे। जब मैं बड़ा हुआ तो पता चला- स्कूल मास्टर के बेटे का, बस उनके भतीजे की, कैंटीन उनके भांजे की और  पढ़ाने वाले भी सब उनके रिश्तेदार ही थे..! \n\nआज मास्टर जी में सरकार में मंत्री हैं.! \n\n#कटाक्ष   
3  हमारे गांव में एक मास्टर जी रहते थे, वो मोहल्ले के बच्चों को अपने स्कूल आने को कहते थे। जब मैं बड़ा हुआ तो पता चला- स्कूल मास्टर के बेटे का, बस उनके भतीजे की, कैंटीन उनके भांजे की और  पढ़ाने वाले भी सब उनके रिश्तेदार ही थे..! \n\nआज मास्टर जी में सरकार में मंत्री हैं.! \n\n#कटाक्ष   
4  हमारे गांव में एक मास्टर जी रहते थे, वो मोहल्ले के बच्चों को अपने स्कूल आने को कहते थे। जब मैं बड़ा हुआ तो पता चला- स्कूल मास्टर के बेटे का, बस उनके भतीजे की, कैंटीन उनके भांजे की और  पढ़ाने वाले भी सब उनके रिश्तेदार ही थे..! \n\nआज मास्टर जी में सरकार में मंत्री हैं.! \n\n#कटाक्ष   

       label  word_count  
0  sarcastic          58  
1  sarcastic          58  
2  sarcastic          58  
3  sarcastic          58  
4  sarcastic          58

In [5]:
# Remove All Emojis from Hindi Text Analysis Data
import re
emoji_pattern = re.compile("["                 
        u"U0001F600-U0001F64F"  # emoticons
        u"U0001F300-U0001F5FF"  # symbols & pictographs
        u"U0001F680-U0001F6FF"  # transport & map symbols
        u"U0001F1E0-U0001F1FF"  # flags (iOS)
        u"U00002500-U00002BEF"  # chinese char
        u"U00002702-U000027B0"
        u"U00002702-U000027B0"
        u"U000024C2-U0001F251"
        u"U0001f926-U0001f937"
        u"U00010000-U0010ffff"
        u"u2640-u2642" 
        u"u2600-u2B55"
        u"u200d"
        u"u23cf"
        u"u23e9"
        u"u231a"
        u"ufe0f"  # dingbats
        u"u3030""]+", flags=re.UNICODE)


for i in range(len(df)):
    original_text = df.loc[i, 'text']
    text_without_emojis = emoji.demojize(original_text)
    df.loc[i, 'text'] = text_without_emojis

count_length()

df.tail(100)



text  \
16079                                                                                                                                                                                                                                                मेडिटेशन से आता है #आत्मबल\nऔर\nआत्मबल से मिलती है #खुशियाँ\n\nआत्मबल के हैं अनेक फायदे:-\n\n#meditate\n#meditation https://t.co/FHFuEhbWY7   
16080                                                                                                                                                                                                                कुछ नहीं है हारने को\n         लेकिन\n#SMILE काफी है हराने को \n:smiling_face_with_smiling_eyes:\n\nमुस्कुराया कर ना :smiling_face_with_open_hands: https://t.co/rkIfmCwQC3   
16081                       कभी भी किसी के #चेहरे की #मुस्कान को #देखकर उसकी #खुशी का #अंदाजा ना लगाए क्यूंकि #आप नहीं जानते कि वो #अंदर तक किस कदर #टूटा हुआ है:crying_face:\n\n#पूछा न #जिंदगी मे किसी ने #दिल का हाल.. \nअब #शहर भर में #जिक्र मेरी #ख़ुदकुशी का हैं..:loudly_crying_face: \nमैं स्तब्ध रह गयी ये खबर जानने के बाद..\n#RIPSushant:loudly_crying_face: https://t.co/pFHerPX9ib   
16082                                             EK OR #SITARA AJ #ASMAAN ME CHALA GAYA\n#जिंदा रहने पर कोई साथ नहीं देता और मरने पर लोग दिखावा करते हैं ।।:frowning_face::frowning_face:\nये भी सिखा सिखा गए तुम\nहां जाते-जाते आज फिर से #हिम्मत के साथ \nमरना भी गलत बता गए तुम ।।:frowning_face::frowning_face:\n#ripsushant #sushantsinghrajpoot #SuicideAwareness #Sad \n@sushantsrajput1   
16083                       कभी भी किसी के #चेहरे की #मुस्कान को #देखकर उसकी #खुशी का #अंदाजा ना लगाए क्यूंकि #आप नहीं जानते कि वो #अंदर तक किस कदर #टूटा हुआ है:crying_face:\n\n#पूछा न #जिंदगी मे किसी ने #दिल का हाल.. \nअब #शहर भर में #जिक्र मेरी #ख़ुदकुशी का हैं..:loudly_crying_face: \nमैं स्तब्ध रह गयी ये खबर जानने के बाद..\n#RIPSushant:loudly_crying_face: https://t.co/pFHerPX9ib   
...                                                                                                                                                                                                                                                                                                                                                                                          ...   
16174                                                                                                                                                  स्वर्गीय श्री सुशांत सिंह राजपूत ने हमें छिछोरे मूवी में काम करके यह सिखाया कि संघर्ष से दूर ना भागे । आत्महत्या ना करें और अब खुद ही आत्महत्या कर ली।\n#ripsushant #ripsushantsinghrajput #ripSSR #fuck2020 #sad https://t.co/WwhrA2iqQF   
16175                                                                      बॉलीवुड एक्टर #सुशांत_सिंह_राजपूत ने #फांसी लगाकर की खुदखुशी-\n#खुदखुशी करने वालो पर मुझे तरस नही आता,\nक्योंकि दुनिया मे #खुशी और #गम दोनों मिलते है जो उसे न झेल सके वो इंसान #बुझदिल होता है!\nसुशांत तुमने सही नहीं किया!\nखैर #अल्लाह घर वालों को सब्र अता करे-आमीन\n#SushantSinghRajput https://t.co/a8pRKR5GZU   
16176                                                                                                                                                                #SushantSinghRajput #Suicide  क्यों?? चमक धमक सिर्फ दिखावा है इस फिल्म इंडस्ट्री का, बाकी सब अंदर से टूटे हुए हैं चाहे फिर वो कितना भी महान अदाकार  क्यों न सबकी जिंदगी सिर्फ दिखावा है सिर्फ घोर दिखावा :thumbs_down: #Sad   
16177  हर हाल में #खुश रहना सीख लो:upside-down_face:\n        उस दिल को क्या #उदास रखना:face_with_monocle:\n               जिसमें मेरा #भोले बसता हो:red_heart::folded_hands:\n\n#जय_भोलेनाथ की :folded_hands::shamrock::hibiscus: :prayer_beads::spiral_shell::place_of_worship:\n\n@Being_Suchita @meShital_ \n@Oye_IShuu_ @sushma_pandey1\n@MansiSi62158538 @SSNaina_ https://t.co/5ouhhtgZjt   
16178                    इतना कुछ पाने के बाद भी अगर इंसान जिदंगी से हार जाए , \n   :face_with_rolling_eyes::una

In [6]:
def processText(text):
    text = text.lower()
    text = re.sub('((www.[^s]+)|(https?://[^s]+))','',text)
    text = re.sub('@[^s]+','',text)
    text = re.sub('[s]+', ' ', text)
    text = re.sub(r'#([^s]+)', r'1', text)
    text = re.sub(r'[.!:?\-\'"/]', '', text)
    text = text.strip('\'"')
    return text

for i in range(len(df)):
    df.loc[i, 'text'] = processText(df['text'][i])

count_length()


In [7]:
# Generating Tokens for Hindi Text Analysis

def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
df['text'] = df['text'].apply(lambda x: tokenization(x))

count_length()
df.tail(100)


text  \
16079                                                                                                                                                                     [मेडिटेशन, से, आता, है, 1]   
16080                                                                                                                                                         [कुछ, नहीं, है, हारने, को\n, लेकिन\n1]   
16081                                                                                                                                                                         [कभी, भी, किसी, के, 1]   
16082                                                                                                                                                                                    [ek, or, 1]   
16083                                                                                                                                                                         [कभी, भी, किसी, के, 1]   
...                                                                                                                                                                                              ...   
16174  [स्वर्गीय, श्री, सुशांत, सिंह, राजपूत, ने, हमें, छिछोरे, मूवी, में, काम, करके, यह, सिखाया, कि, संघर्ष, से, दूर, ना, भागे, ।, आत्महत्या, ना, करें, और, अब, खुद, ही, आत्महत्या, कर, ली, ।, \n1]   
16175                                                                                                                                                                            [बॉलीवुड, एक्टर, 1]   
16176                                                                                                                                                                                            [1]   
16177                                                                                                                                                                              [हर, हाल, में, 1]   
16178                  [इतना, कुछ, पाने, के, बाद, भी, अगर, इंसान, जिदंगी, से, हार, जाए, ,, \n, face, _, with, _, rolling, _, eye, unamu, ed, _, face, तो, उसने, क्या, पाया, ,, और, क्या, वो, खोए, 1]   

               label  word_count  
16079  non_sarcastic           5  
16080  non_sarcastic           6  
16081  non_sarcastic           5  
16082  non_sarcastic           3  
16083  non_sarcastic           5  
...              ...         ...  
16174  non_sarcastic          33  
16175  non_sarcastic           3  
16176  non_sarcastic           1  
16177  non_sarcastic           4  
16178  non_sarcastic          35  

[100 rows x 3 columns]

In [8]:
# Remove ‘\n’ from each token
def remove_newlines(strings_list):
    return [s.replace("\n", "") for s in strings_list]

# Apply the function to the 'text' column
df['text'] = df['text'].apply(remove_newlines)


In [9]:
# Remove Stopwords and Punctuations
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punctuations = ['nn', 'n', '।', '/', '`', '+', '\\', '"', '?', '▁(', '$', '@', '[', '_', "'", '!', ',', ':', '^', '|', ']', '=', '%', '&', '.', ')', '(', '#', '*', '', ';', '-', '}', '|', '"']
to_be_removed = stopwords_hi + punctuations + stopwords_en

for i in range(len(df)):
    df.loc[i, 'text'] = ' '.join([ele for ele in df['text'][i] if ele not in to_be_removed])

count_length()
df.tail(100)


text  \
16079                                                                                                मेडिटेशन आता 1   
16080                                                                                                  हारने लेकिन1   
16081                                                                                                         कभी 1   
16082                                                                                                          ek 1   
16083                                                                                                         कभी 1   
...                                                                                                             ...   
16174  स्वर्गीय सुशांत सिंह राजपूत हमें छिछोरे मूवी काम करके सिखाया संघर्ष दूर भागे आत्महत्या अब खुद आत्महत्या ली 1   
16175                                                                                               बॉलीवुड एक्टर 1   
16176                                                                                                             1   
16177                                                                                                      हर हाल 1   
16178                इतना पाने अगर इंसान जिदंगी हार जाए face rolling eye unamu ed face उसने क्या पाया क्या वो खोए 1   

               label  word_count  
16079  non_sarcastic           3  
16080  non_sarcastic           2  
16081  non_sarcastic           2  
16082  non_sarcastic           2  
16083  non_sarcastic           2  
...              ...         ...  
16174  non_sarcastic          19  
16175  non_sarcastic           3  
16176  non_sarcastic           1  
16177  non_sarcastic           3  
16178  non_sarcastic          20  

[100 rows x 3 columns]

In [10]:
# Remove most frequent unnecessary words from Hindi Text Analysis Data
corpus_list =[]
for i in range(len(df)):
    corpus_list +=df['text'][i]
counter=collections.Counter(corpus_list)

to_remove = ['नेहरू', 'लेते', 'कटाक्ष', 'जय', 'शी', 'अगर', 'मास्टर', 'वो', 'सिगरेट', 'बीवी', 'इश्क़', 'किताब', 'वश', 'पटाकर', 'पिलाकर']
for i in range(len(df)):
    df.loc[i, 'text'] = ''.join([ele for ele in df['text'][i] if ele not in to_remove])
count_length()
df.tail(100)


text  \
16079                                                                                                मेडिटेशन आता 1   
16080                                                                                                  हारने लेकिन1   
16081                                                                                                         कभी 1   
16082                                                                                                          ek 1   
16083                                                                                                         कभी 1   
...                                                                                                             ...   
16174  स्वर्गीय सुशांत सिंह राजपूत हमें छिछोरे मूवी काम करके सिखाया संघर्ष दूर भागे आत्महत्या अब खुद आत्महत्या ली 1   
16175                                                                                               बॉलीवुड एक्टर 1   
16176                                                                                                             1   
16177                                                                                                      हर हाल 1   
16178                इतना पाने अगर इंसान जिदंगी हार जाए face rolling eye unamu ed face उसने क्या पाया क्या वो खोए 1   

               label  word_count  
16079  non_sarcastic           3  
16080  non_sarcastic           2  
16081  non_sarcastic           2  
16082  non_sarcastic           2  
16083  non_sarcastic           2  
...              ...         ...  
16174  non_sarcastic          19  
16175  non_sarcastic           3  
16176  non_sarcastic           1  
16177  non_sarcastic           3  
16178  non_sarcastic          20  

[100 rows x 3 columns]

In [11]:
# Remove least common words/tokens
least_common= [word for word, word_count in Counter(corpus_list).most_common()[:-50:-1]]
for i in range(len(df)):
    df.loc[i, 'text'] = ''.join([ele for ele in df['text'][i] if ele not in least_common])
count_length()

df.tail(10))


text  \
16079                                                                                                मेडिटेशन आता 1   
16080                                                                                                  हारने लेकिन1   
16081                                                                                                         कभी 1   
16082                                                                                                          ek 1   
16083                                                                                                         कभी 1   
...                                                                                                             ...   
16174  स्वर्गीय सुशांत सिंह राजपूत हमें छिछोरे मूवी काम करके सिखाया संघर्ष दूर भागे आत्महत्या अब खुद आत्महत्या ली 1   
16175                                                                                               बॉलीवुड एक्टर 1   
16176                                                                                                             1   
16177                                                                                                      हर हाल 1   
16178                इतना पाने अगर इंसान जिदंगी हार जाए face rolling eye unamu ed face उसने क्या पाया क्या वो खोए 1   

               label  word_count  
16079  non_sarcastic           3  
16080  non_sarcastic           2  
16081  non_sarcastic           2  
16082  non_sarcastic           2  
16083  non_sarcastic           2  
...              ...         ...  
16174  non_sarcastic          19  
16175  non_sarcastic           3  
16176  non_sarcastic           1  
16177  non_sarcastic           3  
16178  non_sarcastic          20  

[100 rows x 3 columns]

In [15]:
df_list = []
font = "/Users/shraddha/Downloads/Devanagari/gargi.ttf"

for text in df['text']:
    words = nltk.word_tokenize(text)
    df_list.extend(words)
dictionary=Counter(df_list)
wordcloud = WordCloud(width = 1000, height = 700,
                background_color ='white',
                min_font_size = 10, font_path= font).generate_from_frequencies(dictionary)
# plot the WordCloud image                      
plt.figure(figsize = (18, 8), facecolor = None)
plt.imshow(wordcloud,interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

